In [1]:
>>> from pathlib import Path
>>> DATA_DIR = Path('~').expanduser().absolute() / 'nessvec-data'
>>> DATA_DIR

PosixPath('/home/hobs/nessvec-data')

In [6]:
>>> import wikipedia

>>> def find_wikipedia_titles(query=[
...         'prosocial Artificial Intelligence',
...         'positive sum game agent',
...         'beneficial Artificial Intelligence'],
...         results=100):
...     queries = [query] if isinstance(query, str) else query
...     titles = []
...     for q in queries: 
...         titles.extend(wikipedia.search(q, results=results))
...     return titles

In [7]:
>>> from tqdm import tqdm
>>> import time

>>> filepath = DATA_DIR / 'wikipedia-prosocial-AI.txt'

>>> def create_wikitext(titles, filepath=filepath):
...     with open(filepath, 'wt') as fout:
...         for title in tqdm(titles):
...             fout.write(f'\n\n# {title}\n')
...             fout.write(
...                 wikipedia.page(
...                     title,
...                     auto_suggest=False).content)
...             time.sleep(.350)
...     return filepath

In [8]:
>>> filepath

PosixPath('/home/hobs/nessvec-data/wikipedia-prosocial-AI.txt')

In [9]:
try:
    assert len(filepath.open().read()) > 10_000
except Exception:
    titles = find_wikipedia_titles()
    filepath = create_wikitext(titles=titles, filepath=filepath)    

In [10]:
!tail -n 5 $filepath


== List of wild ancestors and their domesticated counterparts ==


== References ==

In [11]:
>>> import re
>>> def tokenize(text):
...     r""" Split wikipedia text into tokens (words).
...
...     >>> tokenize("'We're _all_1_.'")
...     ["'", "we're", '_all_1_', '.', "'"]
...     """
...     return re.findall(r"\w+'\w+|\w+|[^'\"\w\s]+|\"|'", text.lower())

>>> def tokenize_row(row):
...     row['tokens'] = tokenize(row['text'])
...     return row

In [12]:
>>> tokenize("'We're _all_1_.'")

["'", "we're", '_all_1_', '.', "'"]

Use the huggingface `dataset` package to load your customized torchtext dataset.

In [13]:
>>> import datasets
>>> dset = datasets.load_dataset(
...     'text',
...     data_files=[str(filepath)])
>>> dset

Using custom data configuration default-0c6d4b74fffdfd41
Reusing dataset text (/home/hobs/.cache/huggingface/datasets/text/default-0c6d4b74fffdfd41/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 63744
    })
})

In [14]:
>>> dset = dset.map(tokenize_row)
>>> dset

Loading cached processed dataset at /home/hobs/.cache/huggingface/datasets/text/default-0c6d4b74fffdfd41/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4/cache-3550ddab30b95e11.arrow


DatasetDict({
    train: Dataset({
        features: ['text', 'tokens'],
        num_rows: 63744
    })
})

In [15]:
>>> from itertools import chain
>>> import pandas as pd
>>> vocab = pd.Series(list(set(chain(*dset['train']['tokens']))))  # <1>
>>> vocab

0            prayers
1        tresnlowski
2                ucb
3          selecting
4             danner
            ...     
43991     blacksmith
43992        whistle
43993          addis
43994         sbarge
43995        golfers
Length: 43996, dtype: object

#### <1> dset['train']['tokens'] is a list of the token lists for each paragraph of text (dataset row)

In [16]:
>>> tok2id = pd.Series(vocab.index.values, index=vocab.values)
>>> tok2id

prayers            0
tresnlowski        1
ucb                2
selecting          3
danner             4
               ...  
blacksmith     43991
whistle        43992
addis          43993
sbarge         43994
golfers        43995
Length: 43996, dtype: int64

In [17]:
WINDOW_FULLWIDTH = 7
WINDOW_HALFWIDTH = 3


In [18]:
def skip_grams(row, hw=WINDOW_HALFWIDTH):
    """ Compute sentence (str) to sliding-window of skip-gram pairs. """
    tokens = row['tokens']
    pairs = []
    for i, tok in enumerate(tokens):
        target = tok2id[tok]
        window = [
            i + j for j in range(-hw, hw + 1, 1)
            if (i + j >= 0) & (i + j < len(tokens)) & (j != 0)
        ]
        pairs += [(target, tok2id[tokens[k]]) for k in window]
    row['skip-grams'] = pairs
    return row

In [19]:
>>> dset = dset.map(skip_grams)
>>> dset

0ex [00:00, ?ex/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'tokens', 'skip-grams'],
        num_rows: 63744
    })
})

In [106]:
>>> for i, row in enumerate(dset['train']['skip-grams']):
...     print(i, row[:4])
...     if i > 2:
...         break

0 []
1 []
2 [[3479, 20467], [20467, 3479]]
3 [[20467, 5683], [20467, 30336], [20467, 24759], [5683, 20467]]


In [107]:
>>> for i, row in enumerate(dset['train']['skip-grams']):
...     print([
...         (vocab[pair[0]], vocab[pair[1]]) 
...         for pair in row[:4]
...         ])
...     if i > 2:
...         break

[]
[]
[('#', 'creativity'), ('creativity', '#')]
[('creativity', 'is'), ('creativity', 'a'), ('creativity', 'phenomenon'), ('is', 'creativity')]


### Example skip-gram for alphabet character grams

A dataset loader needs to have a .data attribute that lists the training set.  
In our case we've created the 'window' key to store the windowized tokens that are used to construct the skip-gram keys.

In [20]:
from itertools import chain
from torch.utils.data import Dataset

class Word2VecDataset(Dataset):
    """ Assumes HuggingFace dataset has been augmented with a 'window' dataset key. """

    def __init__(self, dataset, vocab_size):
        self.dataset = dataset
        self.vocab_size = vocab_size
        self.data = list(chain(*['skip-grams']))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [21]:
>>> import multiprocessing
>>> CPU_CORES = multiprocessing.cpu_count()
>>> CPU_CORES

8

### Hyperperamters to be tuned

In [22]:
>>> EPOCHS = 4
>>> LEARNING_RATE = 0.001
>>> BATCH_SIZE = 2**13
>>> BATCH_SIZE

8192

In [23]:
>>> from torch.utils.data import DataLoader

>>> num_workers=max(CPU_CORES - 1, int(.9 * CPU_CORES))
>>> num_workers

7

In [24]:
>>> dataloader = {'train':
...     DataLoader(
...         Word2VecDataset(
...             dset['train'],
...             vocab_size=len(vocab)),
...         batch_size=BATCH_SIZE,
...         shuffle=True,
...         num_workers=max(CPU_CORES - 1, int(.9 * CPU_CORES))
...     )}

In [25]:
def one_hot_encode(input_id, size):
    vec = torch.zeros(size).float()
    vec[input_id] = 1.0
    return vec

### Instantiate Word2Vec model object

In [26]:
from torch import nn
EMBED_DIM = 100  # <1> Quite small, just for the tutorial

class Word2Vec(nn.Module):
    def __init__(self, vocab_size=len(vocab), embed_dim=EMBED_DIM):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.expand = nn.Linear(embed_dim, vocab_size, bias=False)

    def forward(self, input):
        # Encode input to lower-dimensional representation
        hidden = self.embed(input)
        # Expand hidden layer to predictions
        logits = self.expand(hidden)
        return logits

In [27]:
>>> model = Word2Vec()
model

Word2Vec(
  (embed): Embedding(43996, 100)
  (expand): Linear(in_features=100, out_features=43996, bias=False)
)

In [28]:
>>> import torch
>>> if torch.cuda.is_available():
...     device = torch.device('cuda')
>>> else:
...     device = torch.device('cpu')
>>> device

device(type='cpu')

In [29]:
>>> model.to(device)
>>> model

Word2Vec(
  (embed): Embedding(43996, 100)
  (expand): Linear(in_features=100, out_features=43996, bias=False)
)

### Train Word2Vec language model

In [32]:
for pair in dataloader['train']:
    break
pair

['s', '-', 'r', 'k', 'g', 's', 'p', 'a', 'm', 'i']


['s', '-', 'r', 'k', 'g', 's', 'p', 'a', 'm', 'i']

In [26]:
batch = pd.DataFrame([center.tolist(), context.tolist()]).T
batch.columns = 'center context'.split()
batch

,center,context
0,19853,9892
1,7099,20107
2,7330,14169
3,20107,15029
4,1802,5924
...,...,...
8187,20107,19853
8188,17037,12003
8189,17037,9363
8190,3677,14509


In [27]:
for c in batch.columns:
    batch[c] = vocab[batch[c]].values
batch

,center,context
0,on,a
1,the,","
2,would,this
3,",",for
4,one,scholars
...,...,...
8187,",",on
8188,and,940
8189,and,things
8190,her,remaining


In [28]:
from tqdm import tqdm
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

In [31]:
!ls ~/nessvec-data/models

Word2Vec-state_dict-2022-02-06T14-loss_6.0697712898254395.pt
Word2Vec-state_dict-2022-02-06T16-loss_5.526773929595947.pt
Word2Vec-state_dict-2022-02-06T16-loss_6.8196306228637695.pt
Word2Vec-state_dict-2022-02-06T22-loss_6.991849422454834.pt
Word2Vec-state_dict-2022-02-22T21-loss_6.028685092926025.pt
Word2Vec-state_dict-2022-02-22T22-loss_6.232699871063232.pt
Word2Vec-state_dict-20641x100-2022-02-23T15-loss_6.61333703994751.pt
Word2Vec-state_dict-4943x100-2022-02-22T22-loss_6.232699871063232.pt
Word2Vec-state_dict.pt
Word2Vec_WikiText2_ch06.py


In [30]:
def train_model(model=model, epochs=EPOCHS, dataloader=dataloader, loss_fn=loss_fn):
    losses = []
    pbar = tqdm(range(EPOCHS * len(dataloader['train'])),
                position=0, leave=True)
    for epoch in range(EPOCHS):
        epoch_loss = 0
        for sample_num, (center, context) in enumerate(dataloader['train']):
            center, context = center.to(device), context.to(device)
            optimizer.zero_grad()
            logits = model(input=context)
            loss = loss_fn(logits, center)
    #         if not sample_num % 10:
    #             print(epoch, sample_num, loss.item())
            if not sample_num % 10:
                pbar.set_description(
                    f'loss[{epoch},{sample_num}] = {loss.item()}', 
                    position=0, leave=True)
            epoch_loss += loss.item()
            loss.backward()
            optimizer.step()
            pbar.update(1)
        epoch_loss /= len(dataloader['train'])
        losses.append(epoch_loss)
    return model, losseslo9,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,9,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

  0%|                                                                                                               | 0/1584 [00:00<?, ?it/s]

0 0 10.09812068939209


  0%|                                                                                                       | 1/1584 [00:01<51:56,  1.97s/it]

0 1 10.089266777038574


  0%|▏                                                                                                      | 2/1584 [00:03<40:13,  1.53s/it]

...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████▊| 1582/1584 [28:17<00:01,  1.11it/s]

3 394 6.602845191955566


100%|███████████████████████████████████████████████████████████████████████████████████████████████████▉| 1583/1584 [28:18<00:00,  1.04it/s]

3 395 6.5754170417785645


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 1584/1584 [28:18<00:00,  1.13it/s]

In [ ]:
losses


In [ ]:
import datetime  # noqa

def save_model(model, loss=torch.tensor(float("NaN")), name='Word2Vec', model_dir=Path(DATA_DIR) / 'models'):
    """ Save the model with a unique timestamped filename including shape and loss """
    try:
        loss = loss.item()
    except AttributeError:
        pass
    model_dir.mkdir(exist_ok=True)

    num_vecs, embed_dim = model.state_dict()["embed.weight"].shape
    now = datetime.datetime.now()

    filename = f'{name}-state_dict-{num_vecs}x{embed_dim}-{now.isoformat()[:13]}-loss_{loss}.pt'

    torch.save(model.state_dict(), model_dir / filename)
    return model_dir / filename

In [ ]:
save_model(model, loss)

In [ ]:
def load_model(model, filepath)
    """ Load a model state_dict """
    try:
        loss = loss.item()
    except AttributeError:
        pass
    model_dir.mkdir(exist_ok=True)

    num_vecs, embed_dim = model.state_dict()["embed.weight"].shape
    now = datetime.datetime.now()

    filename = f'{name}-state_dict-{num_vecs}x{embed_dim}-{now.isoformat()[:13]}-loss_{loss}.pt'

    torch.save(model.state_dict(), model_dir / filename)
    return model_dir / filename

In [ ]:
model.state_dict()["expand.weight"].shape

### Learning curve

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()
plt.plot(running_loss)
plt.grid('on')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show(block=False)

In [ ]:
wordvecs = model.expand.weight.cpu().detach().numpy()
tokens = tokenize('mother father woman man dr. writer author white black japanese men women people person')

from scipy.spatial import distance
import numpy as np


def get_distance_matrix(wordvecs, metric):
    dist_matrix = distance.squareform(distance.pdist(wordvecs, metric))
    return dist_matrix


def get_k_similar_words(word, dist_matrix, k=10):
    idx = tok2id[word]
    dists = dist_matrix[idx]
    ind = np.argpartition(dists, k)[:k + 1]
    ind = ind[np.argsort(dists[ind])][1:]
    out = [(i, vocab[i], dists[i]) for i in ind]
    return out


dmat = get_distance_matrix(wordvecs, 'cosine')
for word in tokens:
    print(word, [t[1] for t in get_k_similar_words(word, dmat)], "\n")

In [ ]:
import pandas as pd

pd.Series(tok2id)